# Library Consistency

To check if all downloaded images (files in GDrive folder) have the corresponding metadata saved in the DB and, viceversa, if all metadata in the DB corresponds to existing files.

TO-DO:
    - implement scripts to automatically remove inconsistencies

In [37]:
import sys
sys.path.append("../Library/")
import image_download as imd
import image_manipulation as ima

from dotenv import load_dotenv
import os
load_dotenv("../.env")

import numpy as np
import db_connection

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
IMG_FOLDER = os.environ["MFP_IMG_FOLDER"]

'/Users/peterweber/Google Drive/MFP - Satellogic/images/'

## Images files (GDrive folder)

In [39]:
images_files = ima.get_image_filenames(IMG_FOLDER)

Existing images files: 415


## Images metadata (DB)

In [41]:
db = db_connection.connect("../credentials/mlab_db.txt","mfp")
images_lib_col = db["images_lib"]

DB Credentials from file
DB connected successfully!!!


In [42]:
images_metadata = ima.get_metadata_filenames(images_lib_col)

Existing images metadata: 335


## Missing metadata for file and missing files

In [44]:
missing_metadata, missing_files = ima.get_discrepancies_between_metadata_and_images(images_files, images_metadata)

Missing metadata for 80 files:
 US_2_33.02613505849492_-95.22068486491332_15_600.png
 US_3_37.66401321425672_-97.21715817753926_14_600.png
 US_3_35.399516906014156_-86.419580559389_14_600.png
 US_3_47.39031094307566_-80.9920299256121_16_600.png
 US_4_47.29428238136259_-111.43562902469873_15_600.png
 US_3_42.98806525060572_-110.44960348626935_16_600.png
 US_2_33.02613505849492_-95.22068486491332_16_600.png
 US_5_52.61157891326431_-111.29250648393565_14_600.png
 US_4_34.29070286236522_-106.9256014628645_15_600.png
 US_2_48.9599419343183_-101.5693035409191_16_600.png
 US_3_47.39031094307566_-80.9920299256121_15_600.png
 US_2_36.57250287336959_-112.1243744002766_15_600.png
 US_3_35.399516906014156_-86.419580559389_15_600.png
 US_2_33.02613505849492_-95.22068486491332_13_600.png
 US_5_51.780354429955324_-112.36480570639523_14_600.png
 US_4_41.21642954715827_-122.06630348076318_16_600.png
 US_2_39.32840671980378_-82.51508090746711_14_600.png
 US_2_32.43355025201006_-99.24849463173878_14_600.

## Delete inconsistencies

In [45]:
def delete_image_metadata_inconsistencies(db_collection, img_folder, missing_metadata, missing_files):
    for file in missing_files:
        query = {"filename": file}
        try:
            db_collection.delete_one(query)
            print("Deleted filename", file, "from collection!")
        except:
            print("Filename", file, "not found in collection!")
    for metadata in missing_metadata:
        full_filename = img_folder + metadata
        try:
            os.remove(full_filename)
            print("File", metadata, "removed!")
        except:
            print("Image", metadata, "not found!")
        
delete_image_metadata_inconsistencies(images_lib_col, IMG_FOLDER, missing_metadata, missing_files)

File US_2_33.02613505849492_-95.22068486491332_15_600.png removed!
File US_3_37.66401321425672_-97.21715817753926_14_600.png removed!
File US_3_35.399516906014156_-86.419580559389_14_600.png removed!
File US_3_47.39031094307566_-80.9920299256121_16_600.png removed!
File US_4_47.29428238136259_-111.43562902469873_15_600.png removed!
File US_3_42.98806525060572_-110.44960348626935_16_600.png removed!
File US_2_33.02613505849492_-95.22068486491332_16_600.png removed!
File US_5_52.61157891326431_-111.29250648393565_14_600.png removed!
File US_4_34.29070286236522_-106.9256014628645_15_600.png removed!
File US_2_48.9599419343183_-101.5693035409191_16_600.png removed!
File US_3_47.39031094307566_-80.9920299256121_15_600.png removed!
File US_2_36.57250287336959_-112.1243744002766_15_600.png removed!
File US_3_35.399516906014156_-86.419580559389_15_600.png removed!
File US_2_33.02613505849492_-95.22068486491332_13_600.png removed!
File US_5_51.780354429955324_-112.36480570639523_14_600.png remo